In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

import os
os.getcwd()

'/project/6010096/moenoor/Health'

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from datetime import datetime

# Function to replace rare categories with 'other' in categorical columns
def replace_rare_categories(df, columns, threshold=0.01):
    for col in columns:
        value_counts = df[col].value_counts(normalize=True)
        categories_to_replace = value_counts[value_counts < threshold].index
        df[col] = df[col].apply(lambda x: f'{col}_other' if x in categories_to_replace else x)
    return df

# Function to one-hot encode categorical features and align indices properly
def one_hot_encode(df, categorical_columns):
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    categorical_features = df[categorical_columns]
    encoded_features = encoder.fit_transform(categorical_features)
    encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns), index=df.index)
    df_encoded = pd.concat([df.drop(columns=categorical_columns), encoded_features_df], axis=1)
    return df_encoded

# Function to generate frequency tables for specified columns
def generate_frequency_tables(df, columns):
    frequency_tables = {}
    for col in columns:
        frequency_tables[col] = df[col].value_counts(dropna=False).reset_index()
        frequency_tables[col].columns = [col, 'Frequency']
    return frequency_tables

# Function to calculate time difference, handling missing values
def time_difference(t1, t2):
    if pd.isna(t1) or pd.isna(t2):
        return None
    datetime1 = datetime.combine(datetime.min, t1)
    datetime2 = datetime.combine(datetime.min, t2)
    return (datetime2 - datetime1).seconds / 60  # Difference in minutes

# Load dataset
df = pd.read_csv('./data/full_dataset.csv')
print("Initial shape of the dataset:", df.shape)

# Convert columns to datetime
df['or_entered_dttm'] = pd.to_datetime(df['or_entered_dttm'])
df['or_left_dttm'] = pd.to_datetime(df['or_left_dttm'])

# Extract time portion
df['entered_time'] = df['or_entered_dttm'].dt.time
df['left_time'] = df['or_left_dttm'].dt.time

print(df['actual_casetime_minutes'].describe())

# Apply function to calculate time difference
df['actual_casetime_minutes'] = df.apply(lambda row: time_difference(row['entered_time'], row['left_time']), axis=1)

# Display summary statistics
print(df['actual_casetime_minutes'].describe())

Initial shape of the dataset: (17288, 57)
count     17281.000000
mean        194.208495
std        4533.563741
min           8.000000
25%          70.000000
50%          99.000000
75%         159.000000
max      430640.000000
Name: actual_casetime_minutes, dtype: float64
count    17281.000000
mean       134.878595
std        104.149855
min          8.000000
25%         70.000000
50%         99.000000
75%        158.000000
max        889.000000
Name: actual_casetime_minutes, dtype: float64


In [3]:
# Filter rows based on conditions
# df = df[df['actual_casetime_minutes'] < 8000]
df = df[df['age_at_discharge'].between(18, 130)]
df = df[(df['avg_bmi'].between(5, 200)) | (df['avg_bmi'].isna())]
print("Shape after filtering rows based on conditions:", df.shape)

# Convert date columns to datetime
df['case_date'] = pd.to_datetime(df['case_date'])
df['scheduled_start_dttm'] = pd.to_datetime(df['scheduled_start_dttm'])
df['scheduled_end_dttm'] = pd.to_datetime(df['scheduled_end_dttm'])

# Extract time-related features
df['scheduled_start_hour'] = df['scheduled_start_dttm'].dt.hour
df['scheduled_end_hour'] = df['scheduled_end_dttm'].dt.hour

# Define and apply function to categorize time of day
def get_period(time):
    return 1 if 0 <= time.hour < 12 else 0

# Extract additional date-related features
df['day_of_year'] = df['case_date'].dt.dayofyear
df['day_of_week'] = df['case_date'].dt.dayofweek
df['Morning_scheduled'] = df['scheduled_start_dttm'].apply(get_period)
df['scheduled_duration'] = (df['scheduled_end_dttm'] - df['scheduled_start_dttm']).dt.total_seconds() / 60

# Rename column
df = df.rename(columns={'scheduled_surgeon_id': 'secondary_scheduled_surgeon_id'})

# Convert columns to appropriate types
df['secondary_scheduled_surgeon_id'] = df['secondary_scheduled_surgeon_id'].apply(lambda x: 0 if x == 0 else 1)
df['first_scheduled_case_of_day_status'] = df['first_scheduled_case_of_day_status'].apply(lambda x: 1 if x == 'First Scheduled Case Of Day' else 0)
df['last_scheduled_case_of_day_status'] = df['last_scheduled_case_of_day_status'].apply(lambda x: 1 if x == 'Last Scheduled Case Of Day' else 0)
df['surgical_location_or_service'] = df['surgical_location_or_service'].apply(lambda x: 1 if x == 'General Surgery' else 0)

# Define column groups
numerical_columns = ['scheduled_duration', 'avg_bmi', 'age_at_discharge', 'day_of_year', 'day_of_week', 'asa_score']
binary_columns = ['secondary_scheduled_surgeon_id', 'Morning_scheduled', 'first_scheduled_case_of_day_status', 'last_scheduled_case_of_day_status', 'surgical_location_or_service']
categorical_columns = ['sex', 'case_service', 'surg_encounter_type', 'primary_surgeon_id', 'scheduled_procedure', 'scheduled_procedure_approach', 'surgical_location_hospital', 'most_responsible_dx_category', 'anesthetic_type']

# Keep only the necessary columns
df = df[categorical_columns + binary_columns + numerical_columns + ['actual_casetime_minutes']]
print("Shape after selecting necessary columns:", df.shape)

# Replace rare categories in categorical columns
df = replace_rare_categories(df, categorical_columns)

# Remove all rows with missing values except for 'avg_bmi'
df = df.dropna(subset=[col for col in df.columns if col != 'avg_bmi'])
print("Shape after removing rows with missing values (except 'avg_bmi'):", df.shape)

# Generate frequency tables for categorical and binary columns
frequency_tables = generate_frequency_tables(df, categorical_columns + binary_columns)

# Save intermediate DataFrame
df.to_csv('./data/df_final.csv', index=True)
print("Intermediate DataFrame saved as 'df_final.csv'")

# Check for missing values and create a list of columns with at least one missing value
missing_values_summary = df.isna().sum()
columns_with_missing_values = missing_values_summary[missing_values_summary > 0]

# Display the results
print("Columns with at least one missing value and the count of missing values:")
print(columns_with_missing_values)

# Convert categorical columns to string, keeping NaN values as NaN
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.astype(str).where(x.notna(), None))

# One-hot encode the categorical columns
df = one_hot_encode(df, categorical_columns)
print("Shape after one-hot encoding categorical columns:", df.shape)

# Save the preprocessed DataFrame
df.to_csv('./data/df_preprocessed.csv')
print("Preprocessed DataFrame saved as 'df_preprocessed.csv'")

# Print the frequency tables for categorical and binary variables
print("\nFrequency tables for categorical and binary variables before one-hot encoding:")
for col, freq_table in frequency_tables.items():
    print(f"\nFrequency table for {col}:")
    print(freq_table)


Shape after filtering rows based on conditions: (17258, 59)
Shape after selecting necessary columns: (17258, 21)
Shape after removing rows with missing values (except 'avg_bmi'): (17246, 21)
Intermediate DataFrame saved as 'df_final.csv'
Columns with at least one missing value and the count of missing values:
avg_bmi    5998
dtype: int64
Shape after one-hot encoding categorical columns: (17246, 66)
Preprocessed DataFrame saved as 'df_preprocessed.csv'

Frequency tables for categorical and binary variables before one-hot encoding:

Frequency table for sex:
         sex  Frequency
0     Female       9042
1       Male       8189
2  sex_other         15

Frequency table for case_service:
         case_service  Frequency
0     General Surgery      14374
1      Otolaryngology       2649
2  case_service_other        223

Frequency table for surg_encounter_type:
  surg_encounter_type  Frequency
0        One Day Stay       9116
1  Same Day Admission       7898
2           Inpatient        232

